In [ ]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict

# Define the overall state schema
class MyState(TypedDict):
    key1: int
    key2: int
    key3: str  # will be filled by node A
    key4: str  # will be filled by node B

# Node A: adds something to key3
def node_A(state: MyState):
    # read key1 & key2
    k1 = state["key1"]
    k2 = state["key2"]
    # do some logic
    result3 = f"sum is {k1 + k2}"
    return {"key3": result3}

# Node B: adds something to key4
def node_B(state: MyState):
    k1 = state["key1"]
    k2 = state["key2"]
    # different logic
    result4 = f"product is {k1 * k2}"
    return {"key4": result4}

# (Optional) Aggregator node after both
def aggregator(state: MyState):
    return {"final": f"Results: {state['key3']} | {state['key4']}"}

# Build graph
builder = StateGraph(MyState)
builder.add_node("NodeA", node_A)
builder.add_node("NodeB", node_B)
builder.add_node("Aggregator", aggregator)

# Add edges to run A & B in parallel from START
builder.add_edge(START, "NodeA")
builder.add_edge(START, "NodeB")

# Then both feed into Aggregator
builder.add_edge("NodeA", "Aggregator")
builder.add_edge("NodeB", "Aggregator")

# Then aggregator to END
builder.add_edge("Aggregator", END)

graph = builder.compile()

In [ ]:
# Invoke with initial state
initial = {"key1": 3, "key2": 5, "key3": "", "key4": ""}
result = graph.invoke(initial)
print(result)